In [0]:
table_name = []
temp = []
for i in dbutils.fs.ls('mnt/bronze/'):
    temp.append(i.name.split('_')[1]) #taking the file class, filtering out the name and then splitting the file name out

for i in temp:
    table_name.append(i.split('.')[0]) # splitting just the table name from schema_table

In [0]:
table_name

Out[2]: ['CountryRegionCurrency',
 'CreditCard',
 'Currency',
 'CurrencyRate',
 'Customer',
 'PersonCreditCard',
 'SalesOrderDetail',
 'SalesOrderHeader',
 'SalesOrderHeaderSalesReason',
 'SalesPerson',
 'SalesPersonQuotaHistory',
 'SalesReason',
 'SalesTaxRate',
 'SalesTerritory',
 'SalesTerritoryHistory',
 'ShoppingCartItem',
 'SpecialOffer',
 'SpecialOfferProduct',
 'Store']

In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

# reading the files from the bronze folder and taking all the columns of a table into a list
for i in table_name:
    path = '/mnt/bronze/' + 'Sales_' + i + '.parquet'
    df = spark.read.format('parquet').load(path)
    column = df.columns

#iterating over all the columns in the list, if the column contains the keyword date, we change the format to yyyy-mm-dd
    for col in column:
        if "Date" in col or "date" in col:
            df = df.withColumn(col,date_format(from_utc_timestamp(df[col].cast(TimestampType()),"UTC"), "yyyy-MM-dd"))
            #df = df.withColumn("ModifiedDate",date_format(from_utc_timestamp(df["ModifiedDate"].cast(TimestampType()),"UTC"),"yyyy-MM-dd"))

#once the transformation is done, we then upload all the files to silver layer in the delta format                                                                  
    output_path = '/mnt/silver/' + 'Sales_' + i
    df.write.format('delta').mode("overwrite").save(output_path)

In [0]:
df.limit(10).show() 

+----------------+--------------------+-------------+--------------------+--------------------+------------+
|BusinessEntityID|                Name|SalesPersonID|        Demographics|             rowguid|ModifiedDate|
+----------------+--------------------+-------------+--------------------+--------------------+------------+
|             292|Next-Door Bike Store|          279|<StoreSurvey xmln...|a22517e3-848d-4eb...|  2014-09-12|
|             294|Professional Sale...|          276|<StoreSurvey xmln...|b50ca50b-c601-4a1...|  2014-09-12|
|             296|      Riders Company|          277|<StoreSurvey xmln...|337c3688-1339-4e1...|  2014-09-12|
|             298|  The Bike Mechanics|          275|<StoreSurvey xmln...|7894f278-f0c8-4d1...|  2014-09-12|
|             300|   Nationwide Supply|          286|<StoreSurvey xmln...|c3fc9705-a8c4-4f3...|  2014-09-12|
|             302|Area Bike Accesso...|          281|<StoreSurvey xmln...|368be6dd-30e5-49b...|  2014-09-12|
|             304|B